In [ ]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename));

In [ ]:
import pandas as pd
multiple_choice_responses = pd.read_csv("../input/kaggle-survey-2019/multiple_choice_responses.csv")
other_text_responses = pd.read_csv("../input/kaggle-survey-2019/other_text_responses.csv")
questions_only = pd.read_csv("../input/kaggle-survey-2019/questions_only.csv")
survey_schema = pd.read_csv("../input/kaggle-survey-2019/survey_schema.csv");

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn import svm
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from plotly.offline import init_notebook_mode, iplot 
import plotly.graph_objs as go
import plotly.offline as py
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from sklearn import tree

# <font color='#000000'><b>Analysis of the 2019 Kaggle ML and DS Survey.Russia and USA in machine learning and data science</b></font>
___
![](https://mail.google.com/mail/u/0?ui=2&ik=36b6eec1e5&attid=0.1&permmsgid=msg-a:r-6669950466188283133&th=16ecb5334bf4a6d6&view=att&disp=safe&realattid=f_k3pq6aep0)

Data is the ruler of the new world. Today, your life is being encoded in bits and bytes. And it is not just your browsing history or your social media activities. Your shopping or travel habits, what games you play, what shows you watch, and pretty much anything that can have the possibility of data generation, does have data generation.

Pretty much all industries, small to large scale transactions, economic to weather patterns have data associated with them. So it is safe to say that we are the needle in the data haystack.

Data is currency, as long as we can find value in them.

let's do data analysis with dataset on the state of ML and data science.

 **First idea** is to compare respondents from the USA and Russia and find the depending between the answers of respondents and these countries.

**Second idea** is to build models and find insite in data with features important.

**At the first step**, I built charts and histograms to showing the ratio of men and women in these countries, salary, quantity of money which are  respondents sent on machine learning or cloud computing products at your work in the past 5 years.

**At the second step**, I built models using XGBoost and CatBoost, and using shap for found important features.
I also built a decision tree and visualized it.

**At the third step**, I used features important for visualization and compared the answers to the questionnaire that the model chose as important for classifying respondents by country.

# <font color='#000000'> First step </font> 
## <font color='#000000'> 1.1 Countries distribution </font>
I decided to compare the answers of respondents from Russia and United States of America, so let's look on  distribution in these countries on diagram.

![](http://mail.google.com/mail/u/0?ui=2&ik=36b6eec1e5&attid=0.1&permmsgid=msg-a:r-3527435563257964064&th=16ec711a8f7de7cf&view=att&disp=safe&realattid=f_k3ojo0p61)

In [ ]:
df1 = multiple_choice_responses
df1.drop([0], axis = 0, inplace = True) 
df1 = df1.astype(str)
df1 = df1.apply(le.fit_transform)

df2_1 = df1[(df1.Q2==1)&(df1.Q3==43)]
df2_2 = df1[(df1.Q2==0)&(df1.Q3==43)]
df2_3 = df1[(df1.Q2==1)&(df1.Q3==57)]
df2_4 = df1[(df1.Q2==0)&(df1.Q3==57)]

df2 = pd.concat([df2_1,df2_2,df2_3,df2_4])

df7 = multiple_choice_responses
df7 = df7.astype(str)
df7_1 = df7[(df7.Q2=="Male")&(df7.Q3=="United States of America")]
df7_2 = df7[(df7.Q2=="Female")&(df7.Q3=="United States of America")]
df7_3 = df7[(df7.Q2=="Male")&(df7.Q3=="Russia")]
df7_4 = df7[(df7.Q2=="Female")&(df7.Q3=="Russia")]

df7 = pd.concat([df7_1,df7_2,df7_3,df7_4])

In [ ]:
df10=  df7[(df7.Q3=="Russia")]
df11 = df7[(df7.Q3=="United States of America")]

df10_1 = pd.DataFrame(df10.Q2.value_counts())
df11_1 = pd.DataFrame(df11.Q2.value_counts())
df11_1=df11_1.reset_index()
df10_1=df10_1.reset_index()

df10_1 = df10_1.rename(columns = {'Q2': 'Q2_Russia','index' : 'Index'})
df11_1 = df11_1.rename(columns = {'Q2': 'Q2_USA', 'index' : 'Index'})

result = pd.merge(df10_1,df11_1, how = 'outer', on = 'Index')
result = result.set_index('Index');

In [ ]:
a_rus = df10.Q3.value_counts()
a_usa = df11.Q3.value_counts()

labels = ['Russia','USA']
values = [620,3011]
colors = ['#000000', '#ee544c'] 

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])

fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=0.5)))

fig.show()

* Most of the respondents are from USA (3011 respondents) and minority of the respondents are from Russia ( 620 respondents ).
* This is due to the fact that the data science was extensive development in the United States way before than in Russia.

## <font color='#000000'> 1.2 Gender </font> 
Durind the past two years women in data science on a five times less then men. The gender gap extends across all areas of tech, not just data positions. From cybersecurity, to programmers and developers, women remain outnumbered and underpaid in comparison to their male counterparts.

let's take a look at the gender distribution statistics for 2019 year.

![](https://www.ft.com/paidpost/CBS/gender_differences/img/gender.jpg)

In [ ]:
from plotly.subplots import make_subplots

color = ['#264073',"#ff5050"]
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=['Male','Female'], 
                     values=[543, 77], 
                     name="Russia",
                     marker_colors=color),1, 1)

fig.add_trace(go.Pie(labels=['Male','Female'], 
                     values=[2387, 624], 
                     name="USA",
                     marker_colors=color),1, 2)
fig.update_layout(
    title={"text":"Gender distribution",
        'x':0.5
          },
    font_size=16,
    annotations=[dict(text='Russia', x=0.18, y=1.1, font_size=20, showarrow=False),
                 dict(text='USA', x=0.82, y=1.1, font_size=20, showarrow=False)])
fig.update_traces( 
                  marker=dict( line=dict(color='#000000', width=0.4)))

fig.show()

* In USA had a <font color='#e60000'>8.3%</font> higther female when female in Russia.
* Why are there so few women scientists? Some research findings point to social barriers — including stereotypes, gender bias, and the climate of science and engineering departments in colleges and universities.
* However, we have to spread the word about girls’ and women’s achievements in data science and ML.
* Teach girls that intellectual skills are acquired.
* Encourage high school girls to take calculus, physics, chemistry, computer science, and engineering classes when available.

## <font color='#000000'>1.2 How much can you make as a data scientist in Russia and in USA? </font> 
Data Science jobs are the highest paid in the IT industry. There is a considerable need for Data Scientist. Demand for Data Scientist is on the peak. But what about the difference in salary in Russia and USA ? Let's compare...

![](https://mail.google.com/mail/u/0?ui=2&ik=36b6eec1e5&attid=0.1&permmsgid=msg-a:r2205783397523837084&th=16ec7b825a70e55e&view=att&disp=safe&realattid=f_k3oq5voo1)

In [ ]:
df10=  df7[(df7.Q3=="Russia")]
df11 = df7[(df7.Q3=="United States of America")]

df10_1 = pd.DataFrame(df10.Q10.value_counts())
df11_1 = pd.DataFrame(df11.Q10.value_counts())
df11_1=df11_1.reset_index()
df10_1=df10_1.reset_index()

df10_1 = df10_1.rename(columns = {'Q10': 'Q10_Russia','index' : 'Index'})
df11_1 = df11_1.rename(columns = {'Q10': 'Q10_USA', 'index' : 'Index'})

result = pd.merge(df10_1,df11_1, how = 'outer', on = 'Index')
result = result.set_index('Index');
result = result.drop(["nan"])

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
y_1 = result.Q10_Russia
y_2 = result.Q10_USA

labels = result.index
colors = ['#ee544c', "#dd423e", "#4974b9", "#fae4d9" , "#f2edf1", "#381430", "#264073","#ff5050"]

fig = make_subplots(
    cols = 2, rows = 1,
    column_widths = [0.4, 0.4],
    specs = [[{'type': 'treemap'}, {'type': 'treemap'}]]
)

fig.add_trace(go.Treemap(
    labels = labels,
    parents = ['Russia'] * len(y_1),
    marker_colors=colors,
    values = y_1),col = 1, row = 1)

fig.add_trace(go.Treemap(
    labels = labels,
    parents = ['USA'] * len(y_2),
    marker_colors=colors,
    values = y_2), col = 2, row = 1)
fig.update_layout(
title={"text":"Salary",
        'x':0.5,
       "font_size" : 20
          })
fig.update_traces( 
                  marker=dict( line=dict(color='#000000', width=0.5)))
fig.show()

* We can be observed a big difference in salary.
* Most respondents in Russia receive their salaries in the range from <font color='#e60000'>0</font> to <font color='#e60000'>20.000 </font>dollars. Most respondents in USA receive their salaries in the range from <font color='#e60000'>80.000</font> to <font color='#e60000'>200.000</font> dollars.
* Mainly,this is due to the different salary levels in general in these countries. Including for data science.

## <font color='#000000'>Which depends on salary? </font> 

In O’Reilly’s data science salary report, 45 percent of those surveyed said they hold the title of “data scientist.” Another 31 percent said they are upper management; an engineer, developer, or programmer; or other. In general, the more a data science professional engages in managerial tasks—such as leading team projects, identifying business problems to be solved with analytics, or communicating with external parties—the higher the salary.

![](http://datasciencedegree.wisconsin.edu/wp-content/uploads/2017/09/Screen-Shot-2017-08-28-at-2.38.03-PM-768x627.png)  
Source: O’Reilly Salary Data Science Salary Report, 2016

## <font color='#000000'>1.3 How much money respondents spent in machine learning or cloud computing products? </font> 

In [ ]:
df10=  df7[(df7.Q3=="Russia")]
df11 = df7[(df7.Q3=="United States of America")]

df10_1 = pd.DataFrame(df10.Q11.value_counts())
df11_1 = pd.DataFrame(df11.Q11.value_counts())
df11_1=df11_1.reset_index()
df10_1=df10_1.reset_index()

df10_1 = df10_1.rename(columns = {'Q11': 'Q11_Russia','index' : 'Index'})
df11_1 = df11_1.rename(columns = {'Q11': 'Q11_USA', 'index' : 'Index'})

result = pd.merge(df10_1,df11_1, how = 'outer', on = 'Index')
result = result.set_index('Index');
result = result.drop(["nan"])

In [ ]:
y_Russia = [190,58,83,41,32,14]
y_USA = [613,130,272,325,323,398]
label = ["0 $",
         "1-99 $",
         "100-999 $",
         "1.000-9.999 $",
         "10.000-99.999 $",
         ">100.000 $"]

colors_1 = ['#264073'] * 6

colors_2 = ['#264073'] * 6
colors_2[4] ="#ff5050"
colors_2[5] ="#ff5050"

fig = make_subplots(1, 2,
                    subplot_titles=['Russia', 'USA'])

fig.add_trace(go.Bar(
    x=label,
    y=y_Russia,
    name='Russsia',
    marker_color=colors_1,
    text=y_Russia,
    textposition='outside',
  
),1, 1)
fig.update_layout(barmode='group')

fig.add_trace(go.Bar(
    x=label,
    y=y_USA,
    name='USA',
    marker_color=colors_2,
    textposition='outside',
    text=y_USA
   
),1, 2)
fig.update_layout(barmode='group',)
fig.update_layout(
title={"text":"How much money respondents spent in \
ML or cloud computing products?",
        'x':0.5,
         "font_size" : 15 })
fig.update_traces( 
                  marker=dict( line=dict(color='#000000', width=0.5)))
fig.show()

* In Russia, a small percentage of respondents spent more than 10,000 in machine learning and cloud computing products at work in the past 5 years.
* If you look at second bar chat at the last two columns, you can see that in thr USA a lot os respodents spend more when 10.000 on machine learning.
* It's directly linked to income level of the country.

## <font color='#000000'>1.4 Respondents who spent in machine learning or cloud computing products more than 100,000 dollars...</font>
![](https://saint-petersburg.ru/i/msg/0361/0061/ph32.jpg)

In [ ]:
df10=  df7[(df7.Q3=="Russia")&(df7.Q11=="> $100,000 ($USD)")]
df11=  df7[(df7.Q3=="United States of America")&(df7.Q11=="> $100,000 ($USD)")]

df10.Q2.value_counts(sort=True).values
df10.Q2.value_counts(sort=True).index

df11.Q2.value_counts(sort=True).values
df11.Q2.value_counts(sort=True).index

In [ ]:
color = ["#264073",'#ff5050']

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=['Male','Female'], 
                     values=[14, 0], 
                     name="Russia",
                     marker_colors=color),1, 1)

fig.add_trace(go.Pie(labels=['Male','Female'], 
                     values=[341,57], 
                     name="USA",
                     marker_colors=color),1, 2)
fig.update_layout(
    title_text="",font_size=16,
    annotations=[dict(text='Russia', x=0.18, y=1.1, font_size=20, showarrow=False),
                 dict(text='USA', x=0.82, y=1.1, font_size=20, showarrow=False)])

fig.update_layout(
title={"text":'Respondents who spent in machine learning more than 100,000$',
        'x':0.5,
         "font_size" : 20 })
fig.update_traces( 
                  marker=dict( line=dict(color='#000000', width=0.5)))
fig.show()

* We haven't got a women from Russia who spend more than 100,000 on machine learning or cloud computing products.
* It's directly linked to income level of the country too. And although nowadays women's educational level was advancing, their salaries were lower than men's in some instances.

# <font color='#000000'> Second step. </font>
---
Let's build a models that will classify respondents by two countries Russia and USA. Then let's use it to find  important features for a classification. It  can help us to understand where respondents from Russia and the USA had responded in different ways to the question.  
#  <font color='#000000'> Shap </font>
##  <font color='#000000'>2.1 XGBoost</font> 

In [ ]:
X_data = df2.drop(['Q3'], axis = 1)
y_data = df2['Q3']
X_train, X_test, y_train, y_test = train_test_split(X_data, 
                                                    y_data, 
                                                    test_size=0.3, 
                                                    random_state=17)
X_train = X_train.fillna(0)   
X_test = X_test.fillna(0) 

In [ ]:
import xgboost
import shap

shap.initjs()
model = xgboost.train({"learning_rate": 0.01}, xgboost.DMatrix(X_train, label=y_train), 100)

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

shap.force_plot(explainer.expected_value, shap_values[0,:], X_train.iloc[0,:])

### Lets look on important features:
* **Q12_Part_3** -Who/what are your favorite media sources that report on data science topics? Answer: Reddit.
* **Q16_Part_3** Answer: Which of the following integrated development environments (IDE's) do you use on a regular basis? Answer: PyCharm.
* **Q4** - What is the highest level of formal education that you have attained or plan to attain within the next 2 years?
* **Q12_Part_10** - Who/what are your favorite media sources that report on data science topics? Answer: Slack Communities (ods.ai, kagglenoobs)
* **Q18_Part_2** - What programming languages do you use on a regular basis? Answer: "R" 
* **Q10** -  What is your current yearly compensation?
* **Q16_Part_2** - Which of the following integrated development environments do you use on a regular basis? Answer: RStudio
![](http://)

* **Q15**  - How long have you been writing code to analyze data (at work or at school)?
* **Q1** - Q1 What is your age?

**Shap** values can explain the output of machine learning model.
The above explanation shows features each contributing to push the model output from the base value (the average model output over the training dataset we passed) to the model output. Features pushing the prediction higher are shown in <font color='#ff0000'>red </font>, those pushing the prediction lower are in <font color='#0000ff'>blue </font>.

If we take many explanations such as the one shown above, rotate them 90 degrees, and then stack them horizontally, we can see explanations for an entire dataset (in the notebook this plot is interactive):

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, X_test)

To get an overview of which features are most important for a model we can plot the Shap values of every feature for every sample. The plot below sorts features by the sum of Shap value magnitudes over all samples, and uses Shap values to show the distribution of the impacts each feature has on the model output. The color represents the feature value (red high, blue low). 

In [ ]:
shap.summary_plot(shap_values, X_test,)

We can also just take the mean absolute value of the Shap values for each feature to get a standard bar plot (produces stacked bars for multi-class outputs):

In [ ]:
shap.summary_plot(shap_values,
                  X_test, 
                  plot_type="bar",
                  color = [
                          
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          '#264073',
                          "#264073",
                          '#e60000',
                          '#e60000',
                          '#e60000',
                          "#e60000",
                          "#e60000",
                          ])

### The model selected these features as the most important:
* **Q12_Part_10** - Who/what are your favorite media sources that report on data science topics? Answer: Slack Communities (ods.ai, kagglenoobs)
* **Q10** -  What is your current yearly compensation?
* **Q15**  - How long have you been writing code to analyze data (at work or at school)?
* **Q18_Part_2** - What programming languages do you use on a regular basis? Answer: "R" 
* **Q4** - What is the highest level of formal education that you have attained or plan to attain within the next 2 years?

## <font color='#000000'>2.2 CatBoost</font> 
l let's do the same for СatBoost and look at the difference

![](http://hsto.org/web/614/0d7/6bb/6140d76bb425435db5182f506382f511.png)

In [ ]:
import catboost
from catboost import *
import shap
shap.initjs()
model = CatBoostRegressor(iterations=300, learning_rate=0.1, random_seed=123)
model.fit(X_train, y_train, verbose=False, plot=False)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)
shap.force_plot(explainer.expected_value, shap_values[0,:], X_train.iloc[0,:])

**Lets look on features importants for CatBoost:**
* **Q2** - Q2 What is your gender? 
* **Q18_Part_6** - Q18 What programming languages do you use on a regular basis? Answer : Java
* **Q13_Part_10** - On which platforms have you begun or completed data science courses? Answer:  University Courses
* **Q6** - What is the size of the company where you are employed?
* **Q18_Part_2** - What programming languages do you use on a regular basis? Answer: "R" 
* **Q12_Part_3** -Who/what are your favorite media sources that report on data science topics? Answer: Reddit.
* **Q15** How long have you been writing code to analyze data (at work or at school)?
* **Q18_Part_5** What programming languages do you use on a regular basis? Answer: C++
* **Q1** What is your age?
* **Q16_Part_9** Answer: Which of the following integrated development environments do you use on a regular basis? Answer: Bash.

In [ ]:
shap.summary_plot(shap_values,
                  X_test, 
                  plot_type="bar",
                  color = ["#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          "#264073",
                          '#264073',
                          "#264073",
                          '#e60000',
                          '#e60000',
                          '#e60000',
                          "#e60000",
                          "#e60000",
                          ])


### Then we use CatBoost the model select these features as the most important:

* **Q10** -  What is your current yearly compensation?
* **Q12_Part_10** - What are your favorite media sources that report on data science topics? Answer: **Slack Communities** (ods.ai, kagglenoobs)
* **Q1** - What is your age?
* **Q15** -  How long have you been writing code to analyze data? 
* **Q16_Part_3** -  Which of the following integrated development environments do you use on a regular basis? Answer: **PyCharm.**

### So, the most important fatures for country predictions it's:
* Salary
* Favorite media sources
* Age
* Programming languages
* Integrated development environments 

## <font color='#000000'>2.3 Decision tree</font> </center>
let's look at the featur importants in the decision tree. I use dtreeviz for visualization.

In [ ]:
clf = DecisionTreeClassifier(max_depth=3)
model = clf.fit(X_train,y_train)  

![](https://mail.google.com/mail/u/0?ui=2&ik=36b6eec1e5&attid=0.2&permmsgid=msg-a:r-9206567982738291197&th=16ec87fc49ead841&view=fimg&sz=s0-l75-ft&attbid=ANGjdJ9VecL53dOMVQeVvAx0dK8pUZ0oAS2UTbzBoGf31HsGi8hTzYnsOjcYAWokVroal5wg6qiZtZZpH2xJEuWy3vjcgIwSHuZPlrQry0w0S2cqIFaYVfMexiSc9Vw&disp=emb&realattid=16ec87fbb7a100125fd1)         

### The decision tree chooses the same features as XGBoost and CatBoost:

* **Q12_Part_10** - What are your favorite media sources that report on data science topics? Answer: **Slack Communities** (ods.ai, kagglenoobs)
* **Q10** -  What is your current yearly compensation?
* **Q18_Part_2** - What programming languages do you use on a regular basis? Answer: **"R"**




# <font color='#000000'> Third step</font>
---
## <font color='#000000' >3.1 Slack Communities (ods.ai, kagglenoobs)</font> 

![](https://mail.google.com/mail/u/0?ui=2&ik=36b6eec1e5&attid=0.1&permmsgid=msg-a:r5887982854580034531&th=16ebc8a3d07a3d1c&view=att&disp=safe&realattid=f_k3lmhv8v0)

In [ ]:
df10=  df7[(df7.Q12_Part_10=='Slack Communities (ods.ai, kagglenoobs, etc)')]
df11=  df7[(df7.Q12_Part_10=='nan')]

df10.Q3.value_counts(sort=True).values
df10.Q3.value_counts(sort=True).index;

df11.Q3.value_counts(sort=True).values
df11.Q3.value_counts(sort=True).index;

In [ ]:
color = ["fae4d9",'#ee544c']
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=['Slack Communities','Other'], 
                      values=[241,379], 
                     name="Russia",
                     marker_colors=color),1, 1)

fig.add_trace(go.Pie(labels=['Slack Communities','Other'], 
                      values=[339,2672], 
                     name="USA",
                     marker_colors=color),1, 2)
fig.update_layout(
    title_text="",font_size=16,
    annotations=[dict(text='Russia', x=0.18, y=1.1, font_size=20, showarrow=False),
                 dict(text='USA', x=0.82, y=1.1, font_size=20, showarrow=False)])

fig.update_layout(
title={"text":" What are your favorite media sources that report on data science topics?",
        'x':0.5,
         "font_size" : 20 })
fig.update_traces( 
                  marker=dict( line=dict(color='#000000', width=0.5)))
fig.show()

*  38.9 % of Russians prefer Slack Communities as favorite media sources, but only 11.3%  respondents from USA chosen Slack.
*  People from different countries and backgrounds discuss a various range of topics in these communities. It is a source of valuable information and contacts.
* Russian respondents prefer use Slack Communities as favorite media sources about 3 times more  than respondents from USA.

## <font color='#000000' >3.2 "R" programming languages</font>

![](https://mail.google.com/mail/u/0?ui=2&ik=36b6eec1e5&attid=0.1&permmsgid=msg-a:r6596166617037118262&th=16ebc95d7e15fc04&view=att&disp=safe&realattid=f_k3lmy6j50)

In [ ]:
df10=  df7[(df7.Q18_Part_2=='R')]
df11=  df7[(df7.Q18_Part_2=='nan')]

df10.Q3.value_counts(sort=True).values
df10.Q3.value_counts(sort=True).index

df11.Q3.value_counts(sort=True).values
df11.Q3.value_counts(sort=True).index;

In [ ]:
color = ["#4974b9",'#264073']
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=['"R" ','Other'], 
                     values=[52,568], 
                     name="Russia",
                     marker_colors=color),1, 1)

fig.add_trace(go.Pie(labels=['"R" ','Other'], 
                      values=[995,2016], 
                     name="USA",
                     marker_colors=color),1, 2)
fig.update_layout(
    title_text="",font_size=16,
    annotations=[dict(text='Russia', x=0.18, y=1.1, font_size=20, showarrow=False),
                 dict(text='USA', x=0.82, y=1.1, font_size=20, showarrow=False)])

fig.update_layout(
title={"text":"What programming languages do you use? ",
        'x':0.5,
       "font_size" : 20 
          })
fig.update_traces( 
                  marker=dict( line=dict(color='#000000', width=0.5)))

fig.show()

<a id="1"></a> <br>
### <font color='000000' >Why "R" programming languages so popular?</font>
One reason is that the language and modern statistics grew up together. It is heavily used in analyzing data that is both structured and unstructured. This has made R, the standard language for performing statistical operations. R allows various features that set it apart from other Data Science languages.
Then, of course, there are the things that make it popular with non-statisticians as well:
1. it is high-quality, non-buggy
2. it is free
3. it has a vast collection of contributed packages
4. it is easy to contribute your own packages

### <font color='000000' >Why "R" programming languages so popular in USA?</font>
I think it's becouse the most students in the USA study "R" at universities. 

## <font color='#000000' >3.3 PyCharm</font>

![](https://mail.google.com/mail/u/0?ui=2&ik=36b6eec1e5&attid=0.1&permmsgid=msg-a:r8089893412839070732&th=16ebd578d3a9a256&view=att&disp=safe&realattid=f_k3lui9uk2)

In [ ]:
df10=  df7[(df7.Q16_Part_3==' PyCharm ')]
df11=  df7[(df7.Q16_Part_3=='nan')]

In [ ]:
df10.Q3.value_counts(sort=True).values
df10.Q3.value_counts(sort=True).index;

In [ ]:
df11.Q3.value_counts(sort=True).values
df11.Q3.value_counts(sort=True).index;

In [ ]:
color_1 = ["#381430",'#dd423e']

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=['PyCharm','Other'], 
                      values=[216,404], 
                     name="Russia",
                     marker_colors=color_1),1, 1)

fig.add_trace(go.Pie(labels=['PyCharm','Other'], 
                      values=[539,2472], 
                     name="USA",
                     marker_colors=color_1),1, 2)
fig.update_layout(
    title_text="",font_size=16,
    annotations=[dict(text='Russia', x=0.18, y=1.1, font_size=20, showarrow=False),
                 dict(text='USA', x=0.82, y=1.1, font_size=20, showarrow=False)])

fig.update_layout(
title={"text":"Which of the following IDE's do you use?",
        'x':0.5,
       "font_size" : 20 
          })
fig.update_traces( 
                  marker=dict( line=dict(color='#000000', width=0.5)))
fig.show()

* One third of Russian respondents prefer PyCharm as integrated development environments.
* Its depends of most popular language in Russia Python.

## <font color='#000000'>3.4 Age distribution</font>

![](https://mail.google.com/mail/u/0?ui=2&ik=36b6eec1e5&attid=0.1&permmsgid=msg-a:r-5148686138655139763&th=16ebca1984babba7&view=att&disp=safe&realattid=f_k3lnektg0)

In [ ]:
df10=  df7[(df7.Q3=="Russia")]
df11 = df7[(df7.Q3=="United States of America")]

In [ ]:
df10_1 = pd.DataFrame(df10.Q1.value_counts())
df11_1 = pd.DataFrame(df11.Q1.value_counts())
df11_1=df11_1.reset_index()
df10_1=df10_1.reset_index()

df10_1 = df10_1.rename(columns = {'Q1': 'Q1_Russia','index' : 'Index'})
df11_1 = df11_1.rename(columns = {'Q1': 'Q1_USA', 'index' : 'Index'})

result = pd.merge(df10_1,df11_1, how = 'outer', on = 'Index')

result = result.set_index('Index');

In [ ]:
import plotly.express as px
y_Russia = [86,105,141,126,68,41,20,8,14,6,5]
y_USA = [157,318,628,544,395,265,225,167,139,]
label = ["18-21","22-24","25-29","30-34","35-39","40-44","45-49","50-54","50-69","70+"]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=label,
    y=y_Russia,
    name='Russsia',
    marker_color='#000000',
     textposition='outside',
    text=y_Russia
))
fig.add_trace(go.Bar(
    x=label,
    y=y_USA,
    name='USA',
    marker_color='#ee544c',
     textposition='outside',
    text=y_USA
))

fig.update_layout(
title={"text":'Age distribution',
        'x':0.5,
       "font_size" : 20 
          })
fig.update_traces( 
                  marker=dict( line=dict(color='#000000', width=0.5)))

fig.show()

* Most respondents in Russa 18-34 years old.Few respondents study ML after 40 years old in Russia.
* Most respondents in USA 22-39 years old.

## <font color='#000000'>3. 5 Education distribution </font>

![](http://umom.biz/wp-content/uploads/2018/02/Skolko-stoit-biznes-obrazovanie-800x445.png)

In [ ]:
df10=  df7[(df7.Q3=="Russia")]
df11 = df7[(df7.Q3=="United States of America")]

df10_1 = pd.DataFrame(df10.Q4.value_counts())
df11_1 = pd.DataFrame(df11.Q4.value_counts())
df11_1=df11_1.reset_index()
df10_1=df10_1.reset_index()

df10_1 = df10_1.rename(columns = {'Q4': 'Q4_Russia','index' : 'Index'})
df11_1 = df11_1.rename(columns = {'Q4': 'Q4_USA', 'index' : 'Index'})


result = pd.merge(df10_1,df11_1, how = 'outer', on = 'Index')

result = result.set_index('Index');
result = result.drop(["nan"])

In [ ]:
y_Russia = result.Q4_Russia
y_USA = result.Q4_USA
label = result.index

color_1 = ["#4974b9","#4974b9","#4974b9","#4974b9",
          "#4974b9","#4974b9","#4974b9","#4974b9",]

fig = make_subplots(1, 2,
                    subplot_titles=['Russia', 'USA'])

fig.add_trace(go.Bar(
    x=label,
    y=y_Russia,
    name='Russsia',
    marker_color=color_1,
    text=y_Russia,
    textposition='outside',
  
),1, 1)
fig.update_layout(barmode='group')

fig.add_trace(go.Bar(
    x=label,
    y=y_USA,
    name='USA',
    marker_color=color_1,
    textposition='outside',
    text=y_USA
   
),1, 2)
fig.update_layout(barmode='group',)


fig.update_layout(
title={"text":'Education distribution',
        'x':0.5,
       "font_size" : 20
          })
fig.show()

* A lot of respondents from Russia professional degree and have no formal education pass hight school.
* Most respondents in Russia and USA have Master's degre.

### Does educational background matter?
 First, we took a look at the highest degree achieved by those who hold the title of “data scientist” or a related field.
 
 ![](http://miro.medium.com/max/1250/1*AZX4pan-ABZ4IxCZ-GKuJw.png)
 
We’ve chosen the job titles of data engineer, data analyst, software engineer, machine learning engineer, and data scientist², as these reflect some of the distinct roles we found in our previous articles.

**Data Scientists**

Data scientists have the highest average education level of any of the job titles we examined.
Data scientists have more PhDs than any of the other job titles. However, a PhD is not required for becoming a data scientist; only 20% of data scientists have them.
Advanced degrees (MA or PhD) are held by 75% of data scientists.
Less than 5% of data scientists have only a high school diploma or associates degree.

**Machine Learning, Data, and Software Engineers**

Software and data engineers have more bachelor’s degrees than advanced degrees, while machine learning engineers are more likely to hold advanced degrees.
Machine learning engineers have a similar distribution of education levels to data scientists, but are about 30% less likely to hold a PhD. These results seem roughly in line with a similar study by Stitch Data.
Engineering-focused roles tend to favor bachelor’s degrees with some masters, but very few (<5%) PhDs.
1 in 4 Data engineers have HS diplomas and associates degrees as their highest level of education.

source : https://medium.com/indeed-engineering/where-do-data-scientists-come-from-fc526023ace

## <font color='#000000'>3.6 Cursera</font>

![](https://mail.google.com/mail/u/0?ui=2&ik=36b6eec1e5&attid=0.1&permmsgid=msg-a:r-4154144493677307574&th=16ebcb0fbddba7d8&view=att&disp=safe&realattid=f_k3lo09m90)

In [ ]:
df10=  df7[(df7.Q13_Part_2=='Coursera')]
df11=  df7[(df7.Q13_Part_2=='nan')]

In [ ]:
df10.Q3.value_counts(sort=True).values
df10.Q3.value_counts(sort=True).index;

In [ ]:
df11.Q3.value_counts(sort=True).values
df11.Q3.value_counts(sort=True).index

In [ ]:
color_1 = ["#ff5050",'#264073']

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=['Coursera','Other'], 
                     values=[373,247], 
                     name="Russia",
                     marker_colors=color_1),1, 1)

fig.add_trace(go.Pie(labels=['Coursera','Other'],  
                     values=[1483,1528], 
                     name="USA",
                     marker_colors=color_1),1, 2)
fig.update_layout(
    
    annotations=[dict(text='Russia', x=0.18, y=1.1, font_size=20, showarrow=False),
                 dict(text='USA', x=0.82, y=1.1, font_size=20, showarrow=False)])
fig.update_layout(
title={"text":"On which platforms have you begun or completed data science courses?",
        'x':0.5,
       "font_size" : 20
          })
fig.update_traces( 
                  marker=dict( line=dict(color='#000000', width=0.5)))
fig.show()

* 39% Russian respondents use Cursera as platforms of data science courses didn't even matter that  American online learning platform founded by Stanford professors.
* This may be due to the fact that there are a good data science courses on Rassian language.
* 39% USA respondents are use Cursera.

## 3.7 C++

In [ ]:
df10=  df7[(df7.Q18_Part_5=='C++')]
df11=  df7[(df7.Q18_Part_5=='nan')]

df10.Q3.value_counts(sort=True).values
df10.Q3.value_counts(sort=True).index

df11.Q3.value_counts(sort=True).values
df11.Q3.value_counts(sort=True).index;

In [ ]:
color_1 = ["#ff5050",'#fae4d9']

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=['C++','Other'], 
                     values=[98,522], 
                     name="Russia",
                     marker_colors=color_1),1, 1)

fig.add_trace(go.Pie(labels=['C++','Other'],  
                     values=[261,2750], 
                     name="USA",
                     marker_colors=color_1),1, 2)
fig.update_layout(
    
    annotations=[dict(text='Russia', x=0.18, y=1.1, font_size=20, showarrow=False),
                 dict(text='USA', x=0.82, y=1.1, font_size=20, showarrow=False)])
fig.update_layout(
title={"text":"What programming languages do you use?",
        'x':0.5,
       "font_size" : 20
          })
fig.update_traces( 
                  marker=dict( line=dict(color='#000000', width=0.5)))
fig.show()

* 15.8% of Russian restondents  chose C++ as programming languages what whey you use on a regular basis.
* This is due to the fact that about 20 years ago the C language was taught in Russian Universities mostly for academic purposes. Hence most of people connected with IT knew C more or less. So When C++ came across of course most of people moved to it first and so majority stayed there because there were no actual need to go anywhere else. Moreover, at  those times programming for MS Windows was probably most demanding profession because this OS was probably of at least 90% of whole market. And as well as Russians are quite technically educated people and engineers - there were no problem to grasp C++ and write quite a lot on it.

![](https://mail.google.com/mail/u/0?ui=2&ik=36b6eec1e5&attid=0.2&permmsgid=msg-a:r698911305666619484&th=16ebd295c5242787&view=fimg&sz=s0-l75-ft&attbid=ANGjdJ_-vSx0p9xhqZ0_ebtgCg4r4vWRTdYCyupUTP7rYrYlaip0J4ERtBIpt7Q52JvgNH567M9b5Gk0tYma-RarExtVFMmzq5rDVr7Z5KNqhJ1plREa93gzarDYL9Y&disp=emb&realattid=16ebd294c6455c386b61)